In [14]:
import os
from dotenv import load_dotenv
load_dotenv()
from wandelbots import Instance, MotionGroup, Planner
from wandelbots.types import Pose

In [ ]:
os.environ["CELL_ID"] = "cell"
os.environ["MOTION_GROUP"] = "YOUR_MOTION_GROUP"
os.environ["TCP"] = "Flange"

## 🤖 Show Robot-Pad

In [ ]:
from IPython.display import IFrame
IFrame(src='/cell/robot-pad', width=700, height=600)

## ⚙️ Configure Robot

In [9]:
my_instance = Instance()

my_robot = MotionGroup(
    instance=my_instance,
    cell=os.getenv("CELL_ID"),
    motion_group=os.getenv("MOTION_GROUP"),
    default_tcp=os.getenv("TCP"),
)

## 🔩 Get Robot Props & State

In [ ]:
print(my_robot.tcps())
print([round(j,2) for j in my_robot.current_joints()])
print(my_robot.current_tcp_pose())

## 📝 Perform Pose Transforms

In [ ]:
current_pose = my_robot.current_tcp_pose()
translation = Pose.from_list([0, 0, -500, 0, 0, 0])
target_pose = translation * current_pose
print(f"Current\n{current_pose}\n\nTarget\n{target_pose}")

## 🎢 Plan a Motion

In [ ]:
HOME = [0, -1.571, 1.571, -1.571, -1.571, 0]

planner = Planner(my_robot)
trajectory = [
    planner.jptp(joints=HOME),
    planner.line(pose=target_pose),
    planner.jptp(joints=HOME),
]
plan_result = planner.plan(
    start_joints=my_robot.current_joints(), trajectory=trajectory
)
print("motion id:", plan_result.motion)

## 🏃🏽‍♀️ Execute Motion

In [ ]:
async for state in my_robot.execute_motion_stream_async(motion=plan_result.motion, speed=25, response_rate_ms=500):
    current_location = state.current_location_on_trajectory
    print(f"Current Location: {current_location}")